# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [53]:
import numpy as np
import pandas as pd
from sklearn import datasets
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import xgboost as xgb

In [6]:
df = sklearn.datasets.load_boston()
X = df.data
y = df.target

In [17]:
X_train = X[:int(0.75*X.shape[0]),:]
X_test = X[int(0.75*X.shape[0]):,:]
y_train = y[:int(0.75*X.shape[0])]
y_test = y[int(0.75*X.shape[0]):]

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [18]:
def sdvig(y,z):
    return y-z

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [42]:
base_algorithms_list = []
coefficients_list = []

In [43]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [45]:
base_algorithms_list.append(DecisionTreeRegressor(max_depth=5,random_state=42).fit(X_train,y_train))
coefficients_list.append(0.9)
for i in range(1,50,1):
    print(np.sqrt(sklearn.metrics.mean_squared_error(y_train,gbm_predict(X_train))))
    base_algorithms_list.append(DecisionTreeRegressor(max_depth=5,random_state=42).fit(X_train,y_train-gbm_predict(X_train)))
    coefficients_list.append(0.9)

21.30151314884814
3.063005327278877
1.8831588147786815
1.5701488983498766
1.3605747697229071
1.1496466155316216
0.9415091378641158
0.820899990226986
0.7671698592918005
0.7071315823536443
0.6222194418591952
0.5682319182538943
0.47388121886618756
0.43006481039432154
0.39294775270180105
0.3472403511176381
0.3084442246971999
0.26486893191125205
0.2288989925094332
0.19366323471673014
0.15382010039400643
0.13519990287819594
0.12891812553646104
0.1045830838234726
0.09143973950399135
0.08137398251652106
0.07193946573162305
0.06655823719784106
0.057000075383193086
0.05155223254695945
0.045642274166008225
0.04094440173409176
0.03737379648333037
0.033607769099903394
0.026707970543460063
0.02287417739196962
0.02063925925152307
0.01780289482409977
0.016415906357792113
0.014980284358817
0.012593394670083743
0.012162711552484363
0.011357552138821956
0.01017419387500346
0.009060470541151671
0.008480894971378265
0.007978346931460245
0.0076324117259908755
0.006843202917476223


In [46]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,gbm_predict(X_test)))

5.078581746120434

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [48]:
base_algorithms_list = []
coefficients_list = []
base_algorithms_list.append(DecisionTreeRegressor(max_depth=5,random_state=42).fit(X_train,y_train))
coefficients_list.append(0.9)
for i in range(1,50,1):
    print(np.sqrt(sklearn.metrics.mean_squared_error(y_train,gbm_predict(X_train))))
    base_algorithms_list.append(DecisionTreeRegressor(max_depth=5,random_state=42).fit(X_train,y_train-gbm_predict(X_train)))
    coefficients_list.append(0.9/(1.0+i))

3.5116385715360714
2.4200939073722108
2.0062225657199066
1.8052575199957286
1.6730831800335009
1.5574250270731436
1.4762247621979738
1.411775350235842
1.3490630924629867
1.303472304725146
1.2647753975986
1.2419922311289875
1.2200439952300144
1.1926398246403378
1.1770673476179931
1.1598306828299345
1.1384768543622275
1.1214790032866762
1.1092813218714423
1.0906577591804238
1.0741774556748842
1.0596912770508262
1.0523893190181057
1.0457527813054621
1.0355147152833428
1.0293793494722068
1.0234483153139344
1.0180823464768531
1.0080586516128365
1.0033685181635614
0.9990612946079998
0.9949050167451774
0.986130052064512
0.9796703526020202
0.9767992493648655
0.9727823431844534
0.9687811110710539
0.9655675730624077
0.9620215694625577
0.9574016416871094
0.9551179008876733
0.9500995036831913
0.947464487719663
0.945422105910566
0.9412922881799953
0.938193798211538
0.9357715877844583
0.9332632908462132
0.93046276475705


In [49]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,gbm_predict(X_test)))

4.812550945781193

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [65]:
xgb_reg = xgb.XGBRegressor(random_state=42,n_jobs=-1,n_estimators=500)
xgb_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=-1,
       nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [69]:
for i in np.arange(3,15,1):
    xgb_reg = xgb.XGBRegressor(random_state=42,n_jobs=-1,n_estimators=500,max_depth=i)
    xgb_reg.fit(X_train,y_train)
    print(np.sqrt(sklearn.metrics.mean_squared_error(y_test,xgb_reg.predict(X_test))))

4.584003763656483
4.959724141618364
4.8933035371103575
4.924704436706297
5.328523169792537
5.1588518731299455
5.180388255516322
5.26142880880057
5.0258093512424304
5.094430785732156
5.13176704924001
5.254729646974859


In [66]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test,xgb_reg.predict(X_test)))

4.584003763656483

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [70]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
np.sqrt(sklearn.metrics.mean_squared_error(y_test,lr.predict(X_test)))

8.254979753549161